In [5]:
import numpy as np
#import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss

In [6]:
data = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 878049 entries, 0 to 878048
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Dates       878049 non-null  object 
 1   Category    878049 non-null  object 
 2   Descript    878049 non-null  object 
 3   DayOfWeek   878049 non-null  object 
 4   PdDistrict  878049 non-null  object 
 5   Resolution  878049 non-null  object 
 6   Address     878049 non-null  object 
 7   X           878049 non-null  float64
 8   Y           878049 non-null  float64
dtypes: float64(2), object(7)
memory usage: 60.3+ MB


In [8]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 884262 entries, 0 to 884261
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Id          884262 non-null  int64  
 1   Dates       884262 non-null  object 
 2   DayOfWeek   884262 non-null  object 
 3   PdDistrict  884262 non-null  object 
 4   Address     884262 non-null  object 
 5   X           884262 non-null  float64
 6   Y           884262 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 47.2+ MB


In [9]:
data.duplicated().sum()

2323

In [10]:
data.drop_duplicates(subset=None, keep="first", inplace=True)

In [11]:
filt = data['Resolution'] == 'NONE'
data.loc[filt, 'Resolution'] = 'UNKNOWN'

In [12]:
filt = data['Category'].str.contains('THEFT')
data.loc[filt, 'Category'].unique()

array(['LARCENY/THEFT', 'VEHICLE THEFT'], dtype=object)

In [13]:
data.drop(["Descript", "Resolution"], axis='columns', inplace=True)

In [14]:
data['Year'] = data['Dates'].apply(lambda x: int(x[0:4]))
data['Month'] = data['Dates'].apply(lambda x: int(x[5:7]))
data['Day'] = data['Dates'].apply(lambda x: int(x[8:10]))

data['Hour'] = data['Dates'].apply(lambda x: int(x[10:13]))
data['Min'] = data['Dates'].apply(lambda x: int(x[14:16]))
data


test['Year'] = test['Dates'].apply(lambda x: int(x[0:4]))
test['Month'] = test['Dates'].apply(lambda x: int(x[5:7]))
test['Day'] = test['Dates'].apply(lambda x: int(x[8:10]))

test['Hour'] = test['Dates'].apply(lambda x: int(x[10:13]))
test['Min'] = test['Dates'].apply(lambda x: int(x[14:16]))

In [15]:
def time_of_day(hour):
    if hour >= 0 and hour < 4:
        return 'Late Night'
    elif hour >= 4 and hour < 6:
        return 'Dawn'
    elif hour >= 6 and hour < 12:
        return 'Morning'
    elif hour >= 12 and hour < 16:
        return 'Afternoon'
    elif hour >= 16 and hour < 20:
        return 'Evening'
    elif hour >= 20 and hour <= 23:
        return 'Night'

In [16]:
def business_hours(hours):
    return 8 <= hours <= 18

In [17]:
def third_of_month(day):
    if day >= 1 and day < 10:
        return '1st_third'
    elif day >= 10 and day < 20:
        return '2nd_third'
    elif day >= 20 and day <= 31:
        return '3rd_third'

In [18]:
def weekend(weekday):
    if weekday == "Saturday" or weekday == "Sunday":
        return True
    else:
        return False

In [19]:
def quarter_of_year(month):
    if month >= 1 and month <= 3:
        return '1st_quarter'
    elif month > 3 and month <= 6:
        return '2nd_quarter'
    elif month > 6 and month <= 9:
        return '3rd_quarter'
    elif month > 9 and month <= 12:
        return '4th_quarter'

In [20]:
# def type_of_street_extraction(Address):
#     type_of_street= Address.split(' ')
#     for index in range(len(type_of_street)):
#         cell_value= type_of_street[index]
#         if len(cell_value) == 2 and cell_value not in['of', 'OF', 'US', 'LA']:
#             #ls_street_types.append(cell_value)
#             return cell_value
#             break

In [21]:
data['Time_of_Day'] = data['Hour'].map(time_of_day)
test['Time_of_Day'] = test['Hour'].map(time_of_day)

In [22]:
data['Business Hours'] = data['Hour'].map(business_hours)
data['Business Hours'].value_counts()

test['Business Hours'] = test['Hour'].map(business_hours)
#data['Business Hours'].value_counts()

In [23]:
data['Third_of_Month'] = data['Day'].map(third_of_month)
test['Third_of_Month'] = test['Day'].map(third_of_month)

In [24]:
data['Weekend'] = data['DayOfWeek'].map(weekend)
test['Weekend'] = test['DayOfWeek'].map(weekend)

In [25]:
data['Quarter_of_Year'] = data['Month'].map(quarter_of_year)
test['Quarter_of_Year'] = test['Month'].map(quarter_of_year)

In [26]:
# data['street_type'] = data['Address'].map(type_of_street_extraction)

In [27]:
data["Dates"] = pd.to_datetime(data["Dates"], format="%Y-%m-%d %H:%M:%S")
test["Dates"] = pd.to_datetime(test["Dates"], format="%Y-%m-%d %H:%M:%S")

In [28]:
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
cal = calendar()
holidays = cal.holidays(start=data['Dates'].min(), end=data['Dates'].max())
data['Holiday'] = data['Dates'].dt.date.astype('datetime64').isin(holidays)
test['Holiday'] = test['Dates'].dt.date.astype('datetime64').isin(holidays)

In [29]:
# data['street_type'].unique()

In [30]:
#data.drop([ 'Hour', 'Min', 'Month', 'Day'], axis='columns', inplace=True)
data

,Dates,Category,DayOfWeek,PdDistrict,Address,X,Y,Year,Month,Day,Hour,Min,Time_of_Day,Business Hours,Third_of_Month,Weekend,Quarter_of_Year,Holiday
0,2015-05-13 23:53:00,WARRANTS,Wednesday,NORTHERN,OAK ST / LAGUNA ST,-122.425892,37.774599,2015,5,13,23,53,Night,False,2nd_third,False,2nd_quarter,False
1,2015-05-13 23:53:00,OTHER OFFENSES,Wednesday,NORTHERN,OAK ST / LAGUNA ST,-122.425892,37.774599,2015,5,13,23,53,Night,False,2nd_third,False,2nd_quarter,False
2,2015-05-13 23:33:00,OTHER OFFENSES,Wednesday,NORTHERN,VANNESS AV / GREENWICH ST,-122.424363,37.800414,2015,5,13,23,33,Night,False,2nd_third,False,2nd_quarter,False
3,2015-05-13 23:30:00,LARCENY/THEFT,Wednesday,NORTHERN,1500 Block of LOMBARD ST,-122.426995,37.800873,2015,5,13,23,30,Night,False,2nd_third,False,2nd_quarter,False
4,2015-05-13 23:30:00,LARCENY/THEFT,Wednesday,PARK,100 Block of BRODERICK ST,-122.438738,37.771541,2015,5,13,23,30,Night,False,2nd_third,False,2nd_quarter,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
878044,2003-01-06 00:15:00,ROBBERY,Monday,TARAVAL,FARALLONES ST / CAPITOL AV,-122.459033,37.714056,2003,1,6,0,15,Late Night,False,1st_third,False,1st_quarter,False
878045,2003-01-06 00:01:00,LARCENY/THEFT,Monday,INGLESIDE,600 Block of EDNA ST,-122.447364,37.731948,2003,1,6,0,1,Late Night,False,1st_third,False,1st_quarter,False
878046,2003-01-06 00:01:00,LARCENY/THEFT,Monday,SOUTHERN,5TH ST / FOLSOM ST,-122.403390,37.780266,2003,1,6,0,1,Late Night,False,1st_third,False,1st_quarter,False
878047,2003-01-06 00:01:00,VANDALISM,Monday,SOUTHERN,TOWNSEND ST / 2ND ST,-122.390531,37.780607,2003,1,6,0,1,Late Night,False,1st_third,False,1st_quarter,False


In [31]:
#frequency_of_category = data[['Category', 'DayOfWeek']].groupby(by=["Category"]).count().sort_values(by='DayOfWeek',
#                                                                                                            ascending=False)
#frequency_of_category=frequency_of_category.rename(columns={'DayOfWeek': 'frequency'})
#frequency_of_category['cum_percent'] = 100 * (frequency_of_category['frequency'].cumsum()/
#                                            frequency_of_category['frequency'].sum())
#frequency_of_category.reset_index(inplace=True)
#frequency_of_category

In [32]:
#here, we will filter the data to focus on the categories that contribute for 90% of the outcomes (to follow the pareeto rule),
#so that the classifier has a lower number of labels that result in most of the crimes 
#filtered_categories= frequency_of_category[frequency_of_category['cum_percent'] <= 91]
#filtered_categories

In [33]:
#we will filter now the training data using the categories mentioned above only;
#data = data[data.Category.isin(filtered_categories['Category'])]
#data.reset_index(inplace=True)
#data

In [34]:
le = LabelEncoder()
data['Category_Labels'] = le.fit_transform(data['Category'])
data['DayOfWeek_Labels'] = le.fit_transform(data['DayOfWeek'])
data['PdDistrict_Labels'] = le.fit_transform(data['PdDistrict'])
data['Time_of_Day_Labels'] = le.fit_transform(data['Time_of_Day'])
data['Third_of_Month_Labels'] = le.fit_transform(data['Third_of_Month'])
data['Quarter_of_Year_Labels'] = le.fit_transform(data['Quarter_of_Year'])
#data['street_type_Labels'] = le.fit_transform(data['street_type'])
data['business_hours_labels'] = le.fit_transform(data['Business Hours'])
data['weekend_labels'] = le.fit_transform(data['Weekend'])
#data['holiday_labels'] = le.fit_transform(data['Holiday'])
data

,Dates,Category,DayOfWeek,PdDistrict,Address,X,Y,Year,Month,Day,...,Quarter_of_Year,Holiday,Category_Labels,DayOfWeek_Labels,PdDistrict_Labels,Time_of_Day_Labels,Third_of_Month_Labels,Quarter_of_Year_Labels,business_hours_labels,weekend_labels
0,2015-05-13 23:53:00,WARRANTS,Wednesday,NORTHERN,OAK ST / LAGUNA ST,-122.425892,37.774599,2015,5,13,...,2nd_quarter,False,37,6,4,5,1,1,0,0
1,2015-05-13 23:53:00,OTHER OFFENSES,Wednesday,NORTHERN,OAK ST / LAGUNA ST,-122.425892,37.774599,2015,5,13,...,2nd_quarter,False,21,6,4,5,1,1,0,0
2,2015-05-13 23:33:00,OTHER OFFENSES,Wednesday,NORTHERN,VANNESS AV / GREENWICH ST,-122.424363,37.800414,2015,5,13,...,2nd_quarter,False,21,6,4,5,1,1,0,0
3,2015-05-13 23:30:00,LARCENY/THEFT,Wednesday,NORTHERN,1500 Block of LOMBARD ST,-122.426995,37.800873,2015,5,13,...,2nd_quarter,False,16,6,4,5,1,1,0,0
4,2015-05-13 23:30:00,LARCENY/THEFT,Wednesday,PARK,100 Block of BRODERICK ST,-122.438738,37.771541,2015,5,13,...,2nd_quarter,False,16,6,5,5,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
878044,2003-01-06 00:15:00,ROBBERY,Monday,TARAVAL,FARALLONES ST / CAPITOL AV,-122.459033,37.714056,2003,1,6,...,1st_quarter,False,25,1,8,3,0,0,0,0
878045,2003-01-06 00:01:00,LARCENY/THEFT,Monday,INGLESIDE,600 Block of EDNA ST,-122.447364,37.731948,2003,1,6,...,1st_quarter,False,16,1,2,3,0,0,0,0
878046,2003-01-06 00:01:00,LARCENY/THEFT,Monday,SOUTHERN,5TH ST / FOLSOM ST,-122.403390,37.780266,2003,1,6,...,1st_quarter,False,16,1,7,3,0,0,0,0
878047,2003-01-06 00:01:00,VANDALISM,Monday,SOUTHERN,TOWNSEND ST / 2ND ST,-122.390531,37.780607,2003,1,6,...,1st_quarter,False,35,1,7,3,0,0,0,0


In [35]:

test['DayOfWeek_Labels'] = le.fit_transform(test['DayOfWeek'])
test['PdDistrict_Labels'] = le.fit_transform(test['PdDistrict'])
test['Time_of_Day_Labels'] = le.fit_transform(test['Time_of_Day'])
test['Third_of_Month_Labels'] = le.fit_transform(test['Third_of_Month'])
test['Quarter_of_Year_Labels'] = le.fit_transform(test['Quarter_of_Year'])
#test['street_type_Labels'] = le.fit_transform(test['street_type'])
test['business_hours_labels'] = le.fit_transform(test['Business Hours'])
test['weekend_labels'] = le.fit_transform(test['Weekend'])

In [36]:
# Set training data (drop labels) and training labels
X_train = data.drop("Category_Labels", axis=1).copy()
Y_train = data["Category_Labels"].copy()

In [37]:
data=data.drop(columns=['Category', 'DayOfWeek','PdDistrict','Address', 'Time_of_Day',
                        'Third_of_Month', 'Quarter_of_Year', 
                        'Weekend', 'Business Hours','Dates'])

data

,X,Y,Year,Month,Day,Hour,Min,Holiday,Category_Labels,DayOfWeek_Labels,PdDistrict_Labels,Time_of_Day_Labels,Third_of_Month_Labels,Quarter_of_Year_Labels,business_hours_labels,weekend_labels
0,-122.425892,37.774599,2015,5,13,23,53,False,37,6,4,5,1,1,0,0
1,-122.425892,37.774599,2015,5,13,23,53,False,21,6,4,5,1,1,0,0
2,-122.424363,37.800414,2015,5,13,23,33,False,21,6,4,5,1,1,0,0
3,-122.426995,37.800873,2015,5,13,23,30,False,16,6,4,5,1,1,0,0
4,-122.438738,37.771541,2015,5,13,23,30,False,16,6,5,5,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
878044,-122.459033,37.714056,2003,1,6,0,15,False,25,1,8,3,0,0,0,0
878045,-122.447364,37.731948,2003,1,6,0,1,False,16,1,2,3,0,0,0,0
878046,-122.403390,37.780266,2003,1,6,0,1,False,16,1,7,3,0,0,0,0
878047,-122.390531,37.780607,2003,1,6,0,1,False,35,1,7,3,0,0,0,0


In [38]:
test = test.drop(columns=[ 'DayOfWeek','PdDistrict','Address', 'Time_of_Day',
                        'Third_of_Month', 'Quarter_of_Year', 
                        'Weekend', 'Business Hours','Dates'])

test

,Id,X,Y,Year,Month,Day,Hour,Min,Holiday,DayOfWeek_Labels,PdDistrict_Labels,Time_of_Day_Labels,Third_of_Month_Labels,Quarter_of_Year_Labels,business_hours_labels,weekend_labels
0,0,-122.399588,37.735051,2015,5,10,23,59,False,3,0,5,1,1,0,1
1,1,-122.391523,37.732432,2015,5,10,23,51,False,3,0,5,1,1,0,1
2,2,-122.426002,37.792212,2015,5,10,23,50,False,3,4,5,1,1,0,1
3,3,-122.437394,37.721412,2015,5,10,23,45,False,3,2,5,1,1,0,1
4,4,-122.437394,37.721412,2015,5,10,23,45,False,3,2,5,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
884257,884257,-122.408983,37.751987,2003,1,1,0,1,False,6,3,3,0,0,0,0
884258,884258,-122.425342,37.792681,2003,1,1,0,1,False,6,4,3,0,0,0,0
884259,884259,-122.445418,37.712075,2003,1,1,0,1,False,6,2,3,0,0,0,0
884260,884260,-122.387394,37.739479,2003,1,1,0,1,False,6,0,3,0,0,0,0


In [39]:
data = data.drop(columns =["Holiday"])
data
test = test.drop(columns =["Holiday"])
data

,X,Y,Year,Month,Day,Hour,Min,Category_Labels,DayOfWeek_Labels,PdDistrict_Labels,Time_of_Day_Labels,Third_of_Month_Labels,Quarter_of_Year_Labels,business_hours_labels,weekend_labels
0,-122.425892,37.774599,2015,5,13,23,53,37,6,4,5,1,1,0,0
1,-122.425892,37.774599,2015,5,13,23,53,21,6,4,5,1,1,0,0
2,-122.424363,37.800414,2015,5,13,23,33,21,6,4,5,1,1,0,0
3,-122.426995,37.800873,2015,5,13,23,30,16,6,4,5,1,1,0,0
4,-122.438738,37.771541,2015,5,13,23,30,16,6,5,5,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
878044,-122.459033,37.714056,2003,1,6,0,15,25,1,8,3,0,0,0,0
878045,-122.447364,37.731948,2003,1,6,0,1,16,1,2,3,0,0,0,0
878046,-122.403390,37.780266,2003,1,6,0,1,16,1,7,3,0,0,0,0
878047,-122.390531,37.780607,2003,1,6,0,1,35,1,7,3,0,0,0,0


In [40]:
test

,Id,X,Y,Year,Month,Day,Hour,Min,DayOfWeek_Labels,PdDistrict_Labels,Time_of_Day_Labels,Third_of_Month_Labels,Quarter_of_Year_Labels,business_hours_labels,weekend_labels
0,0,-122.399588,37.735051,2015,5,10,23,59,3,0,5,1,1,0,1
1,1,-122.391523,37.732432,2015,5,10,23,51,3,0,5,1,1,0,1
2,2,-122.426002,37.792212,2015,5,10,23,50,3,4,5,1,1,0,1
3,3,-122.437394,37.721412,2015,5,10,23,45,3,2,5,1,1,0,1
4,4,-122.437394,37.721412,2015,5,10,23,45,3,2,5,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
884257,884257,-122.408983,37.751987,2003,1,1,0,1,6,3,3,0,0,0,0
884258,884258,-122.425342,37.792681,2003,1,1,0,1,6,4,3,0,0,0,0
884259,884259,-122.445418,37.712075,2003,1,1,0,1,6,2,3,0,0,0,0
884260,884260,-122.387394,37.739479,2003,1,1,0,1,6,0,3,0,0,0,0


In [41]:
data['Category_Labels'].unique()

array([37, 21, 16, 36, 35, 20, 25,  1, 38,  4, 32,  8, 12,  7, 30, 27, 34,
       19, 13, 15, 26,  6, 28, 23,  5,  0, 11, 17,  3,  9, 31, 18, 29, 10,
       14,  2, 33, 24, 22])

In [42]:
#we will try to find out here if the distance between the location and the proximity services can affect
#the crime type and hence, help us in classification; so, we will try to calculate the distance between the location
#and the nearest (bank, bar, restaurant, mall, superMarket, night_club, museum, hospital, school, pharmacy, jewlery, Hotels)

In [43]:
# Set training data (drop labels) and training labels
X_train = data.drop("Category_Labels", axis=1).copy()
Y_train = data["Category_Labels"].copy()

In [44]:

# Use these for ML algorithms that can handle categorical data without OHE
mini_train_data, mini_dev_data, mini_train_labels, mini_dev_labels = train_test_split(X_train, 
                                                                                      Y_train,
                                                                                      stratify=Y_train,
                                                                                      test_size=0.3,
                                                                                      random_state=1)

In [45]:

# Random Forest Ensemble
# n_estimators; This is the number of trees you want to build before taking the maximum voting or averages of predictions.
#Higher number of trees give you better performance but makes your code slower.
random_forest = RandomForestClassifier(n_estimators=550, max_depth= 31, max_features='sqrt',
                                       min_samples_leaf=16, min_samples_split=43, 
                                       random_state=1, verbose=1, n_jobs=2)


random_forest.fit(mini_train_data, mini_train_labels)



[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:   57.9s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:  3.9min
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:  9.6min
[Parallel(n_jobs=2)]: Done 550 out of 550 | elapsed: 11.9min finished


RandomForestClassifier(max_depth=31, max_features='sqrt', min_samples_leaf=16,
                       min_samples_split=43, n_estimators=550, n_jobs=2,
                       random_state=1, verbose=1)

In [46]:
pred_probs = random_forest.predict_proba(mini_dev_data)

rf_loss = log_loss(mini_dev_labels, pred_probs)


print('Random Forest Validation Log Loss: ', rf_loss)

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:   10.1s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:   45.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:  1.7min
[Parallel(n_jobs=2)]: Done 550 out of 550 | elapsed:  2.0min finished


Random Forest Validation Log Loss:  2.349488238541452


In [47]:
pred_probs = random_forest.predict_proba(mini_train_data)

rf_loss = log_loss(mini_train_labels, pred_probs)


print('Random Forest train Log Loss: ', rf_loss)

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:   20.7s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:  1.4min
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:  3.4min
[Parallel(n_jobs=2)]: Done 550 out of 550 | elapsed:  4.4min finished


Random Forest train Log Loss:  1.9524775513332344


In [48]:
test

,Id,X,Y,Year,Month,Day,Hour,Min,DayOfWeek_Labels,PdDistrict_Labels,Time_of_Day_Labels,Third_of_Month_Labels,Quarter_of_Year_Labels,business_hours_labels,weekend_labels
0,0,-122.399588,37.735051,2015,5,10,23,59,3,0,5,1,1,0,1
1,1,-122.391523,37.732432,2015,5,10,23,51,3,0,5,1,1,0,1
2,2,-122.426002,37.792212,2015,5,10,23,50,3,4,5,1,1,0,1
3,3,-122.437394,37.721412,2015,5,10,23,45,3,2,5,1,1,0,1
4,4,-122.437394,37.721412,2015,5,10,23,45,3,2,5,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
884257,884257,-122.408983,37.751987,2003,1,1,0,1,6,3,3,0,0,0,0
884258,884258,-122.425342,37.792681,2003,1,1,0,1,6,4,3,0,0,0,0
884259,884259,-122.445418,37.712075,2003,1,1,0,1,6,2,3,0,0,0,0
884260,884260,-122.387394,37.739479,2003,1,1,0,1,6,0,3,0,0,0,0


In [55]:
X_test = test.drop("Id", axis=1).copy()
X_test = X_test.astype(float)
X_test.dtypes


X                         float64
Y                         float64
Year                      float64
Month                     float64
Day                       float64
Hour                      float64
Min                       float64
DayOfWeek_Labels          float64
PdDistrict_Labels         float64
Time_of_Day_Labels        float64
Third_of_Month_Labels     float64
Quarter_of_Year_Labels    float64
business_hours_labels     float64
weekend_labels            float64
dtype: object

In [62]:
X_test

,X,Y,Year,Month,Day,Hour,Min,DayOfWeek_Labels,PdDistrict_Labels,Time_of_Day_Labels,Third_of_Month_Labels,Quarter_of_Year_Labels,business_hours_labels,weekend_labels
0,-122.399588,37.735051,2015.0,5.0,10.0,23.0,59.0,3.0,0.0,5.0,1.0,1.0,0.0,1.0
1,-122.391523,37.732432,2015.0,5.0,10.0,23.0,51.0,3.0,0.0,5.0,1.0,1.0,0.0,1.0
2,-122.426002,37.792212,2015.0,5.0,10.0,23.0,50.0,3.0,4.0,5.0,1.0,1.0,0.0,1.0
3,-122.437394,37.721412,2015.0,5.0,10.0,23.0,45.0,3.0,2.0,5.0,1.0,1.0,0.0,1.0
4,-122.437394,37.721412,2015.0,5.0,10.0,23.0,45.0,3.0,2.0,5.0,1.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
884257,-122.408983,37.751987,2003.0,1.0,1.0,0.0,1.0,6.0,3.0,3.0,0.0,0.0,0.0,0.0
884258,-122.425342,37.792681,2003.0,1.0,1.0,0.0,1.0,6.0,4.0,3.0,0.0,0.0,0.0,0.0
884259,-122.445418,37.712075,2003.0,1.0,1.0,0.0,1.0,6.0,2.0,3.0,0.0,0.0,0.0,0.0
884260,-122.387394,37.739479,2003.0,1.0,1.0,0.0,1.0,6.0,0.0,3.0,0.0,0.0,0.0,0.0


In [75]:
test.shape

(884262, 15)

In [76]:
Y_test_pred = random_forest.predict(X_test)
Y_test_pred
#Y_test_pred = random_forest.predict_proba(X_test)

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:   24.7s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:  1.8min
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:  4.1min
[Parallel(n_jobs=2)]: Done 550 out of 550 | elapsed:  4.9min finished


array([21, 21, 16, ..., 21, 21, 21])

In [ ]:
Y_pred_dummies = pd.get_dummies(Y_test_pred)

In [77]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler()
mini_train_data = sc.fit_transform(X_train)
mini_test_data = sc.transform(X_test)

In [78]:
mini_train_data 

array([[4.35779480e-02, 1.27590107e-03, 1.00000000e+00, ...,
        3.33333333e-01, 0.00000000e+00, 0.00000000e+00],
       [4.35779480e-02, 1.27590107e-03, 1.00000000e+00, ...,
        3.33333333e-01, 0.00000000e+00, 0.00000000e+00],
       [4.43370967e-02, 1.76958398e-03, 1.00000000e+00, ...,
        3.33333333e-01, 0.00000000e+00, 0.00000000e+00],
       ...,
       [5.47523820e-02, 1.38427270e-03, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [6.11383037e-02, 1.39080336e-03, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [5.89560305e-02, 5.80059062e-04, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [84]:


from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0])], remainder='passthrough')
X_submission = np.array(ct.fit_transform(test))

In [83]:
X_submission.shape

()

In [88]:
# X_submission = sc.transform(X_submission)
# y_pred = random_forest.predict(X_submission)
# m = np.max(y_pred, axis=1).reshape(-1, 1)
# predicted = np.array((y_pred == m), dtype='int32')
# col_names = ohe.categories_[0]
# df_submission = pd.DataFrame()
# for i, entry in enumerate(col_names):
#     df_submission[entry] = predicted[:,i]

# df_submission['Id'] = dataset_test['Id']
# df_submission.set_index('Id', inplace=True)
# df_submission

In [61]:
Y_test_pred.shape

(884262,)

In [71]:

m = np.max(Y_test_pred).reshape(-1, 1)
predicted = np.array(( Y_test_pred == m), dtype='int32')
predicted.shape

(1, 884262)

In [72]:
sample_submission = pd.read_csv('sampleSubmission.csv.zip')
col_names = list(sample_submission.columns)
col_names.remove('Id')

In [74]:
X_test

,X,Y,Year,Month,Day,Hour,Min,DayOfWeek_Labels,PdDistrict_Labels,Time_of_Day_Labels,Third_of_Month_Labels,Quarter_of_Year_Labels,business_hours_labels,weekend_labels
0,-122.399588,37.735051,2015.0,5.0,10.0,23.0,59.0,3.0,0.0,5.0,1.0,1.0,0.0,1.0
1,-122.391523,37.732432,2015.0,5.0,10.0,23.0,51.0,3.0,0.0,5.0,1.0,1.0,0.0,1.0
2,-122.426002,37.792212,2015.0,5.0,10.0,23.0,50.0,3.0,4.0,5.0,1.0,1.0,0.0,1.0
3,-122.437394,37.721412,2015.0,5.0,10.0,23.0,45.0,3.0,2.0,5.0,1.0,1.0,0.0,1.0
4,-122.437394,37.721412,2015.0,5.0,10.0,23.0,45.0,3.0,2.0,5.0,1.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
884257,-122.408983,37.751987,2003.0,1.0,1.0,0.0,1.0,6.0,3.0,3.0,0.0,0.0,0.0,0.0
884258,-122.425342,37.792681,2003.0,1.0,1.0,0.0,1.0,6.0,4.0,3.0,0.0,0.0,0.0,0.0
884259,-122.445418,37.712075,2003.0,1.0,1.0,0.0,1.0,6.0,2.0,3.0,0.0,0.0,0.0,0.0
884260,-122.387394,37.739479,2003.0,1.0,1.0,0.0,1.0,6.0,0.0,3.0,0.0,0.0,0.0,0.0


In [52]:
sample_submission = pd.read_csv('sampleSubmission.csv.zip')
sample_submission.iloc[:, 1:] = pd.DataFrame(Y_test_pred, columns=sample_submission.columns[1:])

In [54]:
sample_submission.to_csv('submission_random_forest.csv', index=False)